<a href="https://colab.research.google.com/github/mrrajeev29/Transcription-of-video-file-using-API/blob/main/Rajeev_Transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The assembly API key is initialized.

In [45]:
import json

api_key = ''
file_path = '/content/drive/MyDrive/API.json'

try:
    with open(file_path, 'r') as file_to_read:
        json_data = json.load(file_to_read)
        api_key = json_data["API_KEY"]

except FileNotFoundError:
    print("JSON file not found at the specified path:", file_path)

except KeyError:
    print("API_KEY key not found in the JSON data.")

except json.JSONDecodeError as e:
    print("Error decoding JSON data:", str(e))

Install python library for YouTube

In [46]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing YouTube library from pytube. The user gets 2 options such that it can upload a YouTube video or else it can upload video from the drive to get audio from the video.

In [48]:
print('For Youtube video enter 1 and for drive video enter 0')
n=int(input('ENTER = '))
if n==1:
  link=input('ENTER THE LINK : ')
  from pytube import YouTube
  video=YouTube(link)
  yt=video.streams.get_audio_only()
  k=yt.download()
elif n==0:
  link=input('ENTER THE LINK : ')
  k=link
else:
  print('Sorry wrong input.')

For Youtube video enter 1 and for drive video enter 0
ENTER = 0
ENTER THE LINK : /content/drive/MyDrive/a-dinner-of-my-life-story-dr-abdul-kalam-stories-english-motivational-stories-pebbles-stories.mp4


Uploading the audio data retrieved from the YouTube video and feeding into the model to get the transcript.

In [34]:
import sys
import time
import requests

filename=k

def read_file(filename,chunk_size=5242880):
  with open(filename,'rb') as _file:
    while True:
      data= _file.read(chunk_size)
      if not data:
        break
      yield data
headers={'authorization':api_key}
response=requests.post('https://api.assemblyai.com/v2/upload',headers=headers,data=read_file(filename))
audio_url= response.json()['upload_url']
print(audio_url)

https://cdn.assemblyai.com/upload/6a188b72-6ba2-4079-9d04-754ac83043ec


This step performs authorization and perform transcript input response

In [21]:
import requests
endpoint="https://api.assemblyai.com/v2/transcript"
json={"audio_url":audio_url}
headers={"authorization":api_key,"content_type":"application/json"}
transcript_input_response=requests.post(endpoint,json=json,headers=headers)
print(transcript_input_response.json())

{'id': '64ste8iabw-ceae-419a-9d03-5d14016f5fef', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/e7b906df-feb1-4d69-b903-1a525b0f71b7', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_categories_result': {}, 'language_dete

In [22]:
transcript_id=transcript_input_response.json()["id"]
transcript_id

'64ste8iabw-ceae-419a-9d03-5d14016f5fef'

The transcript output status is reflected. The status reflected can be processing or completed.

In [24]:
import requests
endpoint=f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
headers={"authorization":api_key}
transcript_output_response=requests.get(endpoint,headers=headers)



In [25]:
transcript_output_response.json()["status"]

'completed'

In [26]:
transcript_output_response.json()

{'id': '64ste8iabw-ceae-419a-9d03-5d14016f5fef',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/e7b906df-feb1-4d69-b903-1a525b0f71b7',
 'text': "A dinner of my life. Good morning, sir. Good morning, friends. Raju, why do you look upset today? Sir, some of my classmates make fun of me all the time. But why? Because I am from a poor family. My dear friends, we should study, eat and play together irrespective of our religion and social status. Yes sir. I keep telling them the same thing. Today, I will tell you an incident that happened with me during my school days. I used to live in my hometown, Rameshwaram. In those times, people used to be very strict about their religious beliefs. However, my science teacher Shiv Subramaniam Ayer tried his best to break the social barriers. Even though he was from an orthodox family he encouraged us to study, eat and p

Gives the output of the transcript.

In [27]:
transcript_output_response.json()['text']

"A dinner of my life. Good morning, sir. Good morning, friends. Raju, why do you look upset today? Sir, some of my classmates make fun of me all the time. But why? Because I am from a poor family. My dear friends, we should study, eat and play together irrespective of our religion and social status. Yes sir. I keep telling them the same thing. Today, I will tell you an incident that happened with me during my school days. I used to live in my hometown, Rameshwaram. In those times, people used to be very strict about their religious beliefs. However, my science teacher Shiv Subramaniam Ayer tried his best to break the social barriers. Even though he was from an orthodox family he encouraged us to study, eat and play together irrespective of our religion and social status. He used to spend a lot of time with me and clarify all my doubts. He always used to motivate me for my future. Sir, can you please help me solve this problem? Sure, Abdul. See, this is how you solve it. One day while I

In [30]:
from google.colab import files

Enables user to download the transcript for the corresponding video.

In [50]:
print('If you want to download enter 0 else enter 1 :- ')
m=int(input(''))
if m==0:
  transcript_file_path = 'Transcript'
  try:
    with open(transcript_file_path, 'w') as file1:
      file1.write(transcript_output_response.json()['text'])
    files.download(transcript_file_path)
  except OSError as e:
      print(f"An error occurred while opening the file: {e}")
  print('Downloaded.')
elif m==1:
  print('You do not want to download.Thank You...')
else:
  print('Sorry Wrong input.')

If you want to download enter 0 else enter 1 :- 
0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded.


In [32]:
print('Thanks...')

Thanks...
